In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import requests


def getData(ticker='X:BTCUSD', count=1, timespan='minute', limit=50000):
    api_key = 'api-key'
    to_date = datetime.now() - timedelta(days=7) # remove the minus timedelta
    from_date = to_date - timedelta(days=1) 
    to_date = to_date.strftime('%Y-%m-%d') 
    from_date = from_date.strftime('%Y-%m-%d')
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{count}/{timespan}/{from_date}/{to_date}?adjusted=true&sort=asc&limit={limit}&apiKey={api_key}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data['results'])
    df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
    df.rename(columns={'o': 'Open', 'h': 'High', 'l': 'Low', 'c': 'Close', 'v': 'Volume'}, inplace=True)
    df.drop(columns=['vw', 'n', 't'], inplace=True)
    return df

df = getData() 
#seconddf = getData('X:BTCUSD', 5, 'second')
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go


class TrendStructure:
    def __init__(self, df, initial_balance=10000):
        self.df = df
        self.trend = 0 
        self.valid_high = df.iloc[0]['High']
        self.valid_low = df.iloc[0]['Low']
        self.potential_high = None
        self.potential_low = None
        self.signals = []  
        self.account_balance = initial_balance
        self.trades = [] 
        self.active_trades = {} 
        self.risk_per_trade = 0.01  
        self.trade_id = 0  
        self.entry_points = []
        self.exit_points = []

    def initalDirection(self):
        print("initialDirection")
        initial = self.df[:30]  
        up = 0
        down = 0
        for i in range(len(initial)):
            if initial.iloc[i]['Close'] > initial.iloc[i - 1]['Close']:  
                up += 1
            else:
                down += 1
        if up >= down: 
            print("initialDirection up")
            return 2
        else:
            print("initialDirection down")
            return 1
    
    def structure(self, index):
        print("structure")
        curr_candle = self.df.iloc[index]
        prev_candle = self.df.iloc[index - 1]
        
        if self.trend == 2:  #Uptrend
            if curr_candle['Close'] < prev_candle['Low'] and self.potential_high is None: #start of a swing
                self.potential_high = prev_candle['High']
                self.potential_low = curr_candle['Low']
            if self.potential_high is not None and curr_candle['Low'] < self.potential_low: #deeper swing
                self.potential_low = curr_candle['Low']
            if curr_candle['Close'] < self.valid_low: #change of direction
                self.trend = 1  #Switch to downtrend
                self.valid_high = self.potential_high
                self.potential_high = None
                self.potential_low = None
                self.signals.append(('sell', self.valid_high, self.valid_low))
            if self.potential_high is not None and curr_candle['Close'] > self.potential_high: #end of a swing
                self.valid_high = self.potential_high
                self.valid_low = self.potential_low
                self.potential_low = None
                self.potential_high = None
                self.signals.append(('buy', self.valid_low, self.valid_high))

        if self.trend == 1:  #Downtrend
            if curr_candle['Close'] > prev_candle['High'] and self.potential_low is None: #start of a swing
                self.potential_low = prev_candle['Low']
                self.potential_high = curr_candle['High']
            if self.potential_low is not None and curr_candle['High'] > self.potential_high: #deeper swing
                self.potential_high = curr_candle['High']
            if curr_candle['Close'] > self.valid_high: #change of direction
                self.trend = 2  #Switch to uptrend
                self.valid_low = self.potential_low
                self.potential_high = None
                self.potential_low = None
                self.signals.append(('buy', self.valid_low, self.valid_high))
            if self.potential_low is not None and curr_candle['Close'] < self.potential_low: #end of a swing
                self.valid_low = self.potential_low
                self.valid_high = self.potential_high
                self.potential_low = None
                self.potential_high = None
                self.signals.append(('buy', self.valid_high, self.valid_low))
    
    def execute_signals(self, type, entry, stop, takeProfit, time, trade_size=100):
            print("execute_signals")
            trade = {
                'id': self.trade_id,
                'type': type,
                'entry_price': entry,
                'stoploss': stop,
                'takeProfit': takeProfit,
                'size': trade_size,
                'entry_time': time
            }
            
            self.active_trades[self.trade_id] = trade
            self.entry_points.append(entry)
            self.trade_id += 1
    
    def check_trades(self, index):
        print("check_trades")
        curr_candle = self.df.iloc[index]
        
        for trade_id in list(self.active_trades.keys()):
            trade = self.active_trades[trade_id]
            
            if trade['type'] == 'buy':
                if curr_candle['Low'] <= trade['stoploss']:
                    self.close_trade(trade_id, trade['stoploss'], curr_candle['timestamp'])
                elif curr_candle['High'] >= trade['takeProfit']:
                    self.close_trade(trade_id, trade['takeProfit'], curr_candle['timestamp'])
            elif trade['type'] == 'sell':
                if curr_candle['High'] >= trade['stoploss']:
                    self.close_trade(trade_id, trade['stoploss'], curr_candle['timestamp'])
                elif curr_candle['Low'] <= trade['takeProfit']:
                    self.close_trade(trade_id, trade['takeProfit'], curr_candle['timestamp'])

        for signal in self.signals[:]:  
            trade_type, stoploss, takeProfit = signal
            entry_price = int((stoploss + takeProfit) / 2)
            
            if trade_type == 'buy' and curr_candle['High'] <= entry_price:
                self.execute_signals('buy', entry_price, stoploss, takeProfit, curr_candle['timestamp'])
                self.signals.remove(signal)
            elif trade_type == 'sell' and curr_candle['Low'] >= entry_price:
                self.execute_signals('sell', entry_price, stoploss, takeProfit, curr_candle['timestamp'])
                self.signals.remove(signal)
    
    def close_trade(self, trade_id, exit_price, exit_time):
        print("close_trade")
        trade = self.active_trades.pop(trade_id)
        
        pnl = (exit_price - trade['entry_price']) if trade['type'] == 'buy' else (trade['entry_price'] - exit_price)
        self.account_balance += pnl
        
        closed_trade = {
            'Trade ID': trade_id,
            'Type': trade['type'],
            'Entry Price': trade['entry_price'],
            'Exit Price': exit_price,
            'Net Profit': pnl,
            'Balance After Trade': self.account_balance,
            'Entry Time': trade['entry_time'],
            'Exit Time': exit_time
        }
        
        self.trades.append(closed_trade)
        self.exit_points.append(exit_price)
    
    def plot_results(self):
        fig = go.Figure(data=[go.Candlestick(x=self.df['timestamp'],
                                               open=self.df['Open'],
                                               high=self.df['High'],
                                               low=self.df['Low'],
                                               close=self.df['Close'],
                                               name='Candlestick')])

        for signal in self.signals:
            trade_type, stoploss, takeProfit = signal
            entry_price = int((stoploss + takeProfit) / 2)
            if trade_type == 'buy':
                fig.add_trace(go.Scatter(x=[self.df['timestamp'].iloc[-1]], 
                    y=[entry_price], 
                    mode='markers', 
                    marker=dict(color='green', size=10), 
                    name='Buy Signal'))
            elif trade_type == 'sell':
                fig.add_trace(go.Scatter(x=[self.df['timestamp'].iloc[-1]], 
                    y=[entry_price], 
                    mode='markers', 
                    marker=dict(color='red', size=10), 
                    name='Sell Signal'))

        fig.update_layout(title='Candlestick Chart with Signals',
            xaxis_title='Time',
            yaxis_title='Price',
            xaxis_rangeslider_visible=False)

        fig.show()

    def backtest(self):
        print("backtest")
        balances = []

        self.trend = self.initalDirection()
        print("This is the initial trend:" + str(self.trend))
        
        for i in range(1, len(self.df)):
            print(self.active_trades)
            self.structure(i)
            self.check_trades(i)
            balances.append(self.account_balance)
        
        trade_df = pd.DataFrame(self.trades)
        
        wins = trade_df[trade_df['Net Profit'] > 0]
        losses = trade_df[trade_df['Net Profit'] <= 0]
        win_rate = len(wins) / len(trade_df) if len(trade_df) > 0 else 0
        avg_win = wins['Net Profit'].mean() if not wins.empty else 0
        avg_loss = losses['Net Profit'].mean() if not losses.empty else 0
        
        print(trade_df.to_markdown())
        print("Total Trades:", len(trade_df))
        print("Wins:", len(wins))
        print("Losses:", len(losses))
        print("Win Rate:", round(win_rate * 100, 2), "%")
        print("Average Win:", avg_win)
        print("Average Loss:", avg_loss)
        
        self.plot_results()
        
        return trade_df

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None) 
trend_structure = TrendStructure(df)
trade_results = trend_structure.backtest()

In [87]:
import pandas as pd
import plotly.graph_objects as go

df['timestamp'] = pd.to_datetime(df['timestamp'])

fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'])])

fig.update_layout(title='Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False)

fig.show()

In [89]:
import pandas as pd
import plotly.graph_objects as go

df['timestamp'] = pd.to_datetime(df['timestamp'])

fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'])])

colors = ['green', 'blue', 'orange', 'purple', 'red'] 

for index, row in trade_results.iterrows():
    entry_color = colors[index % len(colors)]
    exit_color = colors[index % len(colors)] 

    fig.add_trace(go.Scatter(
        x=[row['Entry Time']],
        y=[row['Entry Price']],
        mode='markers',
        marker=dict(color=entry_color, size=10, symbol='triangle-up'),
        name=f'Entry {index + 1}'
    ))

    fig.add_trace(go.Scatter(
        x=[row['Exit Time']],
        y=[row['Exit Price']],
        mode='markers',
        marker=dict(color=exit_color, size=10, symbol='triangle-down'),
        name=f'Exit {index + 1}'
    ))

fig.update_layout(title='Candlestick Chart with Entry and Exit Points',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False)

fig.show()